## Introdução
O EP3 tem dois objetivos principais: 

Parte 3.1- Segmentação do objeto de interesse: O objetivo desta etapa é segmentar o  objeto do fundo, produzindo uma imagem binária: 0 para o fundo e 1 para o objeto. Dois tipos de segmentação serão usados: manual, para geração do ground-truth, e automática usando algum algoritmo de sua escolha. 

Parte 3.2- Classificação do objeto de interesse. Ou seja, aplicar PCA e SVM para classificar os objetos entre as classes definidas.

Nas duas etapas, a acurácia das soluções serão calculadas.

## Header 3.2

3.2- Classificação do objeto de interesse. Esses dois objetivos devem ser organizados em 2 seções diferentes ou dois JN diferentes. 

Uso de PCA e SVM para classificar os objetos coletados.

--------------------
Autores: Luan Carlos da Silva Casagrande
Paulo Henrique da Silveira
Yoshio Mori

Bibliotécas necessárias

In [47]:
import pathlib

from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from skimage.transform import resize
import os 
import numpy as np

Function that recursively goes through directories levels and returns all folder names in a specific directory.

In [48]:
def getAllPerLevel(in_path, in_cLevel, in_fLevel):
    dirs = os.listdir(in_path)
    if(in_cLevel<in_fLevel):
        data = []
        for item in dirs:
            data.extend(getAllPerLevel(os.path.join(in_path,item), in_cLevel+1, in_fLevel))
        data = [item.lower() for item in data]
        dirs = np.unique(data)
        return dirs
    else:
        dirs = np.unique(dirs)
        return [item.lower() for item in dirs]

Leitura e organização dos dados

In [49]:


inPath = 'autoSegmentationFeret'

sizeRef = 256

dirFormat = os.path.join(inPath, '{obj_class}\\{lighting}\\{place}\\{background}')

classes = getAllPerLevel(inPath, 0, 0)
lightingTypes = getAllPerLevel(inPath, 0, 1)
places = getAllPerLevel(inPath, 0, 2)

X = []
y = []
for objClassID in range(len(classes)):
    objClass = classes[objClassID]
    dictComb = {}
    for lighting in lightingTypes:
        for place in places:
            backgrounds = getAllPerLevel(os.path.join(inPath, objClass, lighting, place), 0, 0)
            dictBackgrounds = {}
            for background in backgrounds:
                aux = {'obj_class': objClass,
                       'lighting': lighting,
                       'place': place,
                       'background': background}
                pathAux = dirFormat.format(**aux)                
                dirs = os.listdir(pathAux)
                for dirFile in dirs:
                    img = io.imread(os.path.join(pathAux, dirFile))
                    img = resize(img, (sizeRef, sizeRef),
                       anti_aliasing=True)
                    X.append(img.flatten())
                    y.append(objClassID)


Extração das features. 

PCA das features.


In [50]:
n_compoenents = 500

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)

eigenfaces = pca.components_.reshape((n_components, sizeRef, sizeRef))

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)


otimização da SVM usando kernel radial

In [51]:


X_train_pca, X_test_pca = features(comp)
param_grid = {'C': [1e0, 1e1, 1e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e6, 1e6],
              'gamma': [0.00001, 0.0005, 0.0001, 0.0003, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf = clf.fit(X_train_pca, y_train)
#    print("Best estimator found by grid search:")
#    print(clf.best_estimator_)



Classificação, Cálculo das métricas de classificação, e Relatório. 


In [52]:
y_pred = clf.predict(X_test_pca)

print(classification_report(y_test, y_pred, target_names=classes))
print(confusion_matrix(y_test, y_pred, labels=range(len(classes))))
print(classes, comp)

                 precision    recall  f1-score   support

          batom       0.75      0.96      0.84        28
   caixa de cha       0.81      0.68      0.74        19
         caneca       0.71      0.63      0.67        27
         caneta       0.83      0.97      0.89        30
carreteis pesca       0.91      0.97      0.94        33
          garfo       1.00      0.78      0.88        36
          livro       0.94      0.59      0.73        27
       parafuso       0.89      0.93      0.91        27
     tampa 74mm       0.86      1.00      0.92        55
          tenis       0.91      0.88      0.89        24

       accuracy                           0.86       306
      macro avg       0.86      0.84      0.84       306
   weighted avg       0.87      0.86      0.85       306

[[27  1  0  0  0  0  0  0  0  0]
 [ 1 13  2  1  1  0  0  1  0  0]
 [ 1  1 17  1  0  0  0  1  4  2]
 [ 0  0  0 29  0  0  0  1  0  0]
 [ 0  0  0  0 32  0  1  0  0  0]
 [ 6  0  0  0  0 28  0  0  2  0]
 